In [7]:
from utils import scrape_pdf_links, flatten_pdf_links,regex_clean_links, extract_text_from_pdf_link, indivual_news_scrapper, get_longest_one, get_headlines, num_tokens_from_string, add_timestamp_column
from srcapper.llm import get_good_links
from srcapper.main import scrape_websites
from srcapper.data_store import append_to_database, retrieve_records
import pandas as pd

# def scrapper_content(website_links, content_time = 'today'):


/home/ec2-user/research/content-gen-post/src/utils.py:83: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 83 of the file /home/ec2-user/research/content-gen-post/src/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(data)


In [9]:
def scrapper_content(website_links, content_time = 'today'):
    website_links = []

    links = ["https://resources.automotivemastermind.com/","https://www.autodealertodaymagazine.com/"]
    website_links.extend(links)
    df = scrape_websites(website_links)

    df['pdf_links_inside_pages'] = df['links'].apply(lambda links: [scrape_pdf_links(link) for link in links])
    df['pdf_links_inside_pages'] = df['pdf_links_inside_pages'].apply(flatten_pdf_links)
    df['links_final'] = ""
    for i in range(len(df['links'])):
        df['links_final'][i] = df['links'][i]

    df['prompt'] = ""
    for i in range(len(df['links'])):
        domain = df['website_link'][i].replace("www.","").split("https://")[1].split("/")[0].replace(".com","")
        print(domain)
        prompt = f"This are the links from the website: {domain}.com and i want you to check which links are relevant to news/Blogs not generic links of the website like about single cars and return me them in a list"
        df['prompt'][i] = prompt

    df['good_links'] = ""
    for i in range(len(df['links'])):
        print("Gpt : Getting Good Links", i, len(df['pdf_links_inside_pages']))
        if len(df['pdf_links_inside_pages'][i]) == 0:
            df['good_links'][i] = get_good_links(str(df['links_final'][i]),df['prompt'][i])
        else:
            df['good_links'][i] = []


    df['final_links'] = ""
    for i in range(len(df['good_links'])):
        if df['good_links'][i] is not None:
            if "sorry" not in df['good_links'][i]:
                print(i)
                if len(df['pdf_links_inside_pages'][i]) == 0:
                    df['final_links'][i] = regex_clean_links(df['good_links'][i])
                else:
                    df['final_links'][i] = []
                
    # df['final_links'] = df['final_links'].apply(lambda x: x[:2]) ###

    df_news = pd.DataFrame(columns=['website_link', 'news_link', 'news_text'])
    for i in range(len(df['final_links'])):
        if len(df['final_links'][i]) == 0:
            for link in df['pdf_links_inside_pages'][i]:
                df_news = pd.concat([df_news,pd.DataFrame([{'website_link': df['website_link'][i], 'news_link': link, 'news_text': extract_text_from_pdf_link(link)}])], ignore_index=True)
    else:
        for link in df['final_links'][i]:
            if "search" or "auth" not in link.split("/"):
                # print(link, ":", df['website_link'][i])
                df_news = pd.concat([df_news,pd.DataFrame([{'website_link': df['website_link'][i], 'news_link': link, 'news_text': indivual_news_scrapper(link)}])], ignore_index=True)

    for i in range(len(df_news['news_text'])):
        if len(df_news['news_text'][i]) < 10:
            # drop the row
            df_news.drop(i, inplace=True)

    df_news['news_text_split'] = df_news['news_text'].apply(lambda x: x.lower().split("follow"))
    df_news['news_text_split_final'] = df_news['news_text_split'].apply(lambda x: get_longest_one(x))
    df_news['headlines'] = df_news['news_link'].apply(lambda x : get_headlines(x))

    df_news['news_text_split_final'] =  df_news['headlines'] + ":" + df_news['news_text_split_final']
    df_news['news_text_len'] = df_news['news_text'].apply(lambda x : num_tokens_from_string(x))

    df_news = add_timestamp_column(df_news)
    df_news = df_news.drop("news_text_split", axis=1)
    append_to_database(df_news)
    result_today = retrieve_records(date_range=content_time)
    return result_today

0
1


In [22]:
from sqlalchemy import create_engine
import datetime as dt

def append_to_database(new_df, db_path='sqlite:///news_data.db', table_name='news_data'):
    engine = create_engine(db_path, echo=False)
    with engine.connect() as connection:
        existing_df = pd.read_sql_table(table_name, con=connection)
    existing_df['timestamp'] = pd.to_datetime(existing_df['timestamp'])
    
    today_date = dt.date.today()
    existing_records_today = existing_df[existing_df['timestamp'].dt.date == today_date]
    print(f"Found {len(existing_records_today)} records for today.")
    if existing_records_today.empty:
        new_df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f"Appended {len(new_df)} records to database.")
    elif len(existing_records_today) < 5:
        new_df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f"Appended {len(new_df)} records to database, because there were only {len(existing_records_today)} records for today.")
    else:
        print(f"Found {len(existing_records_today)} records for today. Skipping append.")

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.document import Document
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from md2pdf.core import md2pdf
from utils import zip_folder
from decouple import config
import uuid 
import os

OPEN_API_KEY = config("OPEN_API_KEY")

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",api_key = OPEN_API_KEY)
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please provide detailed summary of each document covering every important points.
Helpful Answer:"""
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

def get_text_chunks_langchain(text):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

def chain_summarizer(text):
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)

    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
    )

    reduce_documents_chain = ReduceDocumentsChain(
        combine_documents_chain=combine_documents_chain,
        collapse_documents_chain=combine_documents_chain,
        token_max=4000,
    )

    map_reduce_chain = MapReduceDocumentsChain(
        llm_chain=map_chain,
        reduce_documents_chain=reduce_documents_chain,
        document_variable_name="docs",
        return_intermediate_steps=False,
    )

    split_docs = get_text_chunks_langchain(text)

    final_summary = map_reduce_chain.run(split_docs)
    return final_summary

def content_gen(text,content_type):
    prompt = ChatPromptTemplate.from_template('''Context:You are a content creator who generates content for social media sites,blogs, etc in markdown format.
        Input:  I want to write a {type}. Here is the summarized description of the context on which you have to generate content.
        Description: {description}
        Consider this while generating the {type}:
        If relevant, include:
        - Emojis to highlight any word
        - Calls to action (to encourage engagement)
        - Hashtags (for social media posts) in the end
        Note:
        - Write in a clear and concise manner, avoiding jargon and unnecessary complexity.
        - Use active voice and strong verbs to keep the content engaging.
        - Check for grammar errors, typos, and factual inaccuracies.
        - Edit for clarity and flow.
        - if any unnesary information is present remove it, because some of it is scrapped content.

        Note: Generate the output in Markdown format
        Output:''')

    model = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0.5, api_key= OPEN_API_KEY)
    output_parser = StrOutputParser()

    chain = prompt | model | output_parser
    content = chain.invoke({"description":text,"type":content_type})
    return content

def daily_news_summary(list_daily_news):
  news = ""
  for i in range(len(list_daily_news)):
    news = news + " " + list_daily_news[i]
  final_summary = chain_summarizer(news)
  return final_summary

def summarized_daily_news_blog(summary,content_type="blog"):
  blog = content_gen(summary,content_type)
  os.makedirs("Summarized_dailyBlog",  exist_ok=True)

  pdf_file_path = os.getcwd()+"/Summarized_dailyBlog/ "+str(uuid.uuid4())+".pdf"
  md2pdf(pdf_file_path,
       md_content=blog,
       md_file_path=None,
       css_file_path=None,
       base_url=None)
  return pdf_file_path

def daily_post_single(list_daily_news,content_type="LinkedIn Post"):

  for i,news in enumerate(list_daily_news):
    single_post = content_gen(news,content_type)
    os.makedirs("Single_dailyPosts",  exist_ok=True)
    folder = os.getcwd()+"/Single_dailyPosts"
    pdf_file_path = os.getcwd()+"/Single_dailyPosts/"+"pdf"+str(i) + ".pdf"
    md2pdf(pdf_file_path,
        md_content=single_post,
        md_file_path=None,
        css_file_path=None,
        base_url=None)

  zip_file_path = os.getcwd()+"/Single_dailyPosts/dailyPost.zip"
  zip_folder(folder, zip_file_path)
  return zip_file_path

def weekly_news_summary(list_weekly_news):
  news = ""
  for i in range(len(list_weekly_news)):
    news = news + " " + list_weekly_news[i]
  final_summary = chain_summarizer(news)
  return final_summary

def weekly_news_blog(summary,content_type="LinkedIn Post"):
  post = content_gen(summary,content_type)
  pdf_file_path = os.getcwd()+"/WeeklyBlog/abc.pdf"
  md2pdf(pdf_file_path,
       md_content=post,
       md_file_path=None,
       css_file_path=None,
       base_url=None)
  return pdf_file_path

def rephrase(content):
  prompt = ChatPromptTemplate.from_template("""
  '''Context: I'm a content creator working on a project. I've generated a piece of content, and I need your help to polish it.
        Input: {data}
        Rephrase the content and improve it while focusing on:
        1. SEO & Readability
        3. Plagarism
        2. Tone & Voice
        3. Action Verbs
        4. Freshness & Relevance
        Note: Generate the content in Markdown format

    Output:
    """)

  model = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0.5, api_key= OPEN_API_KEY)
  output_parser = StrOutputParser()

  chain = prompt | model | output_parser
  content = chain.invoke({"data":content})
  return content


-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'pango-1.0-0': pango-1.0-0: cannot open shared object file: No such file or directory.  Additionally, ctypes.util.find_library() did not manage to locate a library called 'pango-1.0-0'

In [17]:
result_today = retrieve_records(date_range='week')

In [25]:
result_today

[{'timestamp': 1704995344635,
  'news_text': 'CRACK THE CODE:\n4 Opportunities in the Dealership Service Drive \nWith growing aftermarket opportunities, the service drive is a gold mine for dealers to boost their loyalty \nefforts – and conquest new customers. \nBy taking a targeted approach to working the service drive, dealerships can proactively identify and \nengage these customers with personalized interactions.\nThese service-to-sales opportunities fall into two basic categories: customers who currently service with \nyou and those who don’t. The average age of light vehicles on the road reached a record high of  \n12.5 years in 2023 , according to S&P Global Mobility. The volume of vehicles  \nages 6-14 will grow by another 10 million units by 2028  – adding to an \nalready favorable volume of vehicles in the aftermarket target range.\nService Loyal: Cultivating Lifelong Advocates \nService Loyal customers form the bedrock of \ndealership success. \nMastermind data indicates tha